# Google Gemini Integration Test

This notebook tests the Gemini API integration for the Venâncio RPA project.

## 1. Setup and Configuration

In [1]:
import os
from dotenv import load_dotenv
import google.generativeai as genai

# Load environment variables
load_dotenv()

# Get API key
api_key = os.getenv('GEMINI_API_KEY')
print(f"API Key loaded: {api_key[:20]}..." if api_key else "No API key found!")

# Configure Gemini
genai.configure(api_key=api_key)

API Key loaded: AIzaSyAhlA51XpZ_rshb...


## 2. List Available Models

In [2]:
# List all available models
print("Available Gemini models that support generateContent:")
print("="*70)

for model in genai.list_models():
    if 'generateContent' in model.supported_generation_methods:
        print(f"- {model.name}")

Available Gemini models that support generateContent:
- models/gemini-2.5-pro-preview-03-25
- models/gemini-2.5-flash-preview-05-20
- models/gemini-2.5-flash
- models/gemini-2.5-flash-lite-preview-06-17
- models/gemini-2.5-pro-preview-05-06
- models/gemini-2.5-pro-preview-06-05
- models/gemini-2.5-pro
- models/gemini-2.0-flash-exp
- models/gemini-2.0-flash
- models/gemini-2.0-flash-001
- models/gemini-2.0-flash-exp-image-generation
- models/gemini-2.0-flash-lite-001
- models/gemini-2.0-flash-lite
- models/gemini-2.0-flash-lite-preview-02-05
- models/gemini-2.0-flash-lite-preview
- models/gemini-2.0-pro-exp
- models/gemini-2.0-pro-exp-02-05
- models/gemini-exp-1206
- models/gemini-2.0-flash-thinking-exp-01-21
- models/gemini-2.0-flash-thinking-exp
- models/gemini-2.0-flash-thinking-exp-1219
- models/gemini-2.5-flash-preview-tts
- models/gemini-2.5-pro-preview-tts
- models/learnlm-2.0-flash-experimental
- models/gemma-3-1b-it
- models/gemma-3-4b-it
- models/gemma-3-12b-it
- models/gemma-

## 3. Initialize Gemini Model

In [3]:
# Initialize the model
model = genai.GenerativeModel('gemini-2.5-flash')
print("Model initialized: gemini-2.5-flash")

Model initialized: gemini-2.5-flash


## 4. Test 1: Sentiment Analysis

In [4]:
# Test sentiment analysis
prompt = """Analise o sentimento desta frase e retorne JSON:
{
  "sentiment": "Negativo/Neutro/Positivo",
  "score": 0-10,
  "reasoning": "explicação"
}

Responda APENAS com o JSON."""

text = "Produto péssimo, quebrou em 2 dias! Nunca mais compro aqui."

print("Test 1: Sentiment Analysis")
print("="*70)
print(f"Text: {text}")
print()

full_prompt = f"{prompt}\n\nTexto:\n{text}"
response = model.generate_content(full_prompt)

print("Response:")
print(response.text)

Test 1: Sentiment Analysis
Text: Produto péssimo, quebrou em 2 dias! Nunca mais compro aqui.

Response:
```json
{
  "sentiment": "Negativo",
  "score": 1,
  "reasoning": "A frase contém termos fortemente negativos como 'péssimo' e descreve um evento de falha crítica ('quebrou em 2 dias'). A expressão 'Nunca mais compro aqui' reforça a insatisfação extrema e a intenção de não repetir a compra, indicando um sentimento de rejeição e frustração muito alto."
}
```


## 5. Test 2: Classification

In [ ]:
# Test classification
prompt = """Classifique esta reclamação em categorias:
- produto
- atendimento
- entrega
- preco
- outros

Retorne JSON:
{
  "categories": ["cat1", "cat2"],
  "primary_category": "principal"
}

Responda APENAS com o JSON."""

text = "Comprei uma geladeira e ela chegou com defeito. O atendimento ao cliente também foi horrível."

print("Test 2: Classification")
print("="*70)
print(f"Text: {text}")
print()

full_prompt = f"{prompt}\n\nTexto:\n{text}"
response = model.generate_content(full_prompt)

print("Response:")
print(response.text)

## 6. Test 3: Entity Extraction

In [ ]:
# Test entity extraction
prompt = """Extraia entidades desta reclamação e retorne JSON:
{
  "produto": "nome do produto",
  "loja": "nome da loja",
  "funcionario": "nome do funcionário ou null"
}

Responda APENAS com o JSON."""

text = "Comprei um fogão 5 bocas na loja do Shopping Center. O atendente Carlos foi muito rude."

print("Test 3: Entity Extraction")
print("="*70)
print(f"Text: {text}")
print()

full_prompt = f"{prompt}\n\nTexto:\n{text}"
response = model.generate_content(full_prompt)

print("Response:")
print(response.text)

## 7. Test with Real Complaint Data

In [ ]:
# Test with a realistic complaint
complaint = """Comprei um fogão de 5 bocas há 3 meses e já apresentou defeito. 
Liguei no atendimento 4 vezes e ninguém resolve meu problema. 
A atendente Maria disse que ia enviar um técnico mas já faz 2 semanas e nada. 
Estou muito insatisfeito e vou processar a loja se não resolverem!"""

print("Test with Real Complaint")
print("="*70)
print(f"Complaint: {complaint}")
print()

# Combined analysis
combined_prompt = """Analise esta reclamação e retorne um JSON completo com:
{
  "sentiment": {
    "label": "Negativo/Neutro/Positivo",
    "score": 0-10
  },
  "categories": ["produto", "atendimento", etc],
  "primary_category": "principal",
  "entities": {
    "produto": "nome do produto ou null",
    "loja": "nome da loja ou null",
    "funcionario": "nome do funcionário ou null"
  },
  "urgency_indicators": ["lista de palavras que indicam urgência"]
}

Responda APENAS com o JSON."""

full_prompt = f"{combined_prompt}\n\nReclamação:\n{complaint}"
response = model.generate_content(full_prompt)

print("Combined Analysis Response:")
print(response.text)

## 8. Parse JSON Response

In [ ]:
import json

# Clean and parse the response
result_text = response.text.strip()

# Remove markdown formatting if present
if result_text.startswith("```json"):
    result_text = result_text.replace("```json", "").replace("```", "").strip()
elif result_text.startswith("```"):
    result_text = result_text.replace("```", "").strip()

# Parse JSON
try:
    parsed = json.loads(result_text)
    print("Parsed JSON:")
    print(json.dumps(parsed, indent=2, ensure_ascii=False))
except json.JSONDecodeError as e:
    print(f"JSON parsing error: {e}")
    print(f"Raw text: {result_text}")

## 9. Summary

✅ **Tests Completed:**
- Sentiment Analysis
- Classification
- Entity Extraction
- Combined Analysis
- JSON Parsing

**Next Steps:**
1. Run full validation: `python backend/validate_analysis.py`
2. Test API endpoints: `uvicorn backend.app.main:app --reload`
3. Check backend integration